In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
from datetime import datetime
import numpy as np

In [2]:
nombre_tabla='Largest_banks'
nombre_DB='Banks.db'
atributos=['Name', 'MC_USD_Billion']
atributos_final=['Name', 'MC_USD_Billion','MC_GBP_Billion','MC_EUR_Billion','MC_INR_Billion']
archivo_csv= 'Largest_banks_data.csv'
url= 'https://en.wikipedia.org/wiki/List_of_largest_banks'
log_file="code_log.txt"

In [3]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 
    now = datetime.now()
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [4]:
def extract(url_, atributos_):
    
    pagina=requests.get(url_).text   
    soup= BeautifulSoup(pagina,"html.parser")
    data_=pd.DataFrame(columns=atributos_)
    
    filas=soup.find_all('table')[0].find_all('tbody')[0].find_all('tr')
    
    for i,fila in enumerate(filas):
        if i>0:
            col = fila.find_all('td')        
            Name = col[1].find_all('a')[1].get('title')
            MC_USD_Billion = float(col[2].contents[0][:-1])

            new_data= pd.DataFrame([[Name,MC_USD_Billion]],columns=atributos_)
            data_= pd.concat([data_, new_data],ignore_index=True) 
            
    return (data_)


In [5]:
log_progress('Inicio extracción')
data=extract(url,atributos)

log_progress('Fin extracción')
data.head(10)

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [6]:
def Transform(data_inicial,atributos_final_):
    conv=pd.read_csv('exchange_rate.csv')
    EUR=float(conv['Rate'][0])
    GBP=float(conv['Rate'][1])
    INR=float(conv['Rate'][2]) 
    
    n_round=2
    
    data_final=pd.DataFrame(columns=atributos_final_)
    
    for i, filas in data_inicial.iterrows():
        if i!=0:
            data_plus=pd.DataFrame([[filas['Name'],filas['MC_USD_Billion'],np.round((filas['MC_USD_Billion']*GBP),n_round),np.round(filas['MC_USD_Billion']*EUR,n_round),np.round(filas['MC_USD_Billion']*INR,n_round)]],columns=atributos_final_)
            data_final=pd.concat([data_final,data_plus], ignore_index=True)
            
    
    return data_final    

In [7]:
log_progress('Inicio Transformación')

data_2=Transform(data,atributos_final)

log_progress('Fin Transformación')
data_2.head(10)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,Bank of America,231.52,185.22,215.31,19204.58
1,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
2,Agricultural Bank of China,160.68,128.54,149.43,13328.41
3,HDFC Bank,157.91,126.33,146.86,13098.63
4,Wells Fargo,155.87,124.70,144.96,12929.42
5,HSBC,148.90,119.12,138.48,12351.26
6,Morgan Stanley,140.83,112.66,130.97,11681.85
7,China Construction Bank,139.82,111.86,130.03,11598.07
8,Bank of China,136.81,109.45,127.23,11348.39


In [8]:
def load(obj,datos):
    datos.to_csv(obj)

In [9]:
log_progress("Fase de carga iniciada") 
load(archivo_csv, data_2)
log_progress("Fase de carga finalizada") 

In [10]:
pd.read_csv(archivo_csv)

,Unnamed: 0,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,0,Bank of America,231.52,185.22,215.31,19204.58
1,1,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
2,2,Agricultural Bank of China,160.68,128.54,149.43,13328.41
3,3,HDFC Bank,157.91,126.33,146.86,13098.63
4,4,Wells Fargo,155.87,124.70,144.96,12929.42
5,5,HSBC,148.90,119.12,138.48,12351.26
6,6,Morgan Stanley,140.83,112.66,130.97,11681.85
7,7,China Construction Bank,139.82,111.86,130.03,11598.07
8,8,Bank of China,136.81,109.45,127.23,11348.39


In [11]:
def load_to_db(data_,nombre_DB_,nombre_tabla_):
    conn=sqlite3.connect(nombre_DB_)

    data_.to_sql(nombre_tabla, conn, if_exists = 'replace', index =False)

In [12]:
log_progress("Fase de carga a DB iniciada") 
load_to_db(data_2, nombre_DB, nombre_tabla)
log_progress("Fase de carga a DB finalizada") 

In [13]:
def run_queries(query_statement, nombre_DB_,nombre_tabla_):
    conn=sqlite3.connect(nombre_DB_)     
    query_output = pd.read_sql(query_statement, conn)
    print(query_statement)
    print(query_output)

In [14]:
query= f"SELECT * FROM {nombre_tabla}"
run_queries(query,nombre_DB,nombre_tabla)

SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                          Bank of America          231.52          185.22   
1  Industrial and Commercial Bank of China          194.56          155.65   
2               Agricultural Bank of China          160.68          128.54   
3                                HDFC Bank          157.91          126.33   
4                              Wells Fargo          155.87          124.70   
5                                     HSBC          148.90          119.12   
6                           Morgan Stanley          140.83          112.66   
7                  China Construction Bank          139.82          111.86   
8                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          215.31        19204.58  
1          180.94        16138.75  
2          149.43        13328.41  
3          146.86        13098.63  
4          

In [15]:
conn=sqlite3.connect(nombre_DB)   
conn.close()